In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.lda_model
import pyLDAvis.gensim_models

In [ ]:
# re-creating LDA model from notebook 04
df = pd.read_csv('../data/reddit_preprocessed.csv')

# Creating count-vectorizer for pre-processed data
count_text_vectorizer = CountVectorizer(stop_words='english', min_df=5, max_df=0.7)
count_text_vectors = count_text_vectorizer.fit_transform(df["processed_text"])

# Creating TF-IDF vectorizier for pre-processed data
tfidf_text_vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=0.7)
tfidf_text_vectors = tfidf_text_vectorizer.fit_transform(df['processed_text'])

# Latent Dirichlet Allocation (LDA) model
lda_text_model = LatentDirichletAllocation(n_components = 2, random_state=509)
W_lda_text_matrix = lda_text_model.fit_transform(count_text_vectors)
H_lda_text_matrix = lda_text_model.components_

In [ ]:
# Creating interactive visualization for LDA Model
lda_display = pyLDAvis.lda_model.prepare(lda_text_model, count_text_vectors, count_text_vectorizer, sort_topics=False)
pyLDAvis.display(lda_display)

/Users/mtc/Library/Python/3.12/lib/python/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=91950) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/mtc/Library/Python/3.12/lib/python/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=91950) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/mtc/Library/Python/3.12/lib/python/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=91950) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/mtc/Library/Python/3.12/lib/python/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=91950) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Users/mtc/Library/Python/3.12/lib/python/site-packages/jobl